In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os

In [12]:
import os
import pickle
import joblib
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# ————— Paths to your saved artifacts —————
DL_MODEL_PATH     = 'dl_model.keras'
STAT_MODEL_PATH   = 'stat_model.joblib'
TFIDF_PATH        = 'tfidf_vectorizer.joblib'
TOKENIZER_PATH    = 'tokenizer.pkl'

# ————— Load your deep-learning model —————
dl_model = load_model(DL_MODEL_PATH)

# ————— Load your statistical model + vectorizer —————
stat_model = joblib.load(STAT_MODEL_PATH)
tfidf       = joblib.load(TFIDF_PATH)

# ————— Load your Keras tokenizer —————
with open(TOKENIZER_PATH, 'rb') as f:
    tokenizer = pickle.load(f)

# ————— Sequence length (same as during training) —————
MAX_LEN = 300

def classify_essay(text: str) -> tuple[str, float]:
    # 1) Stat model probability for “human”
    stat_p_human = stat_model.predict_proba(tfidf.transform([text]))[0, 1]

    # 2) DL model probability for “human”
    seq    = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    dl_p_human = float(dl_model.predict(padded, verbose=0)[0, 0])

    # 3) Combined probability
    p_human = (stat_p_human + dl_p_human) / 2

    if p_human > 0.5:
        return "Human", p_human
    else:
        return "AI", 1 - p_human

# ————— Quick sanity-check on a sample essay —————


sample = '''
    9th January 2025
The Evolution of Citizenship
Citizenship represents a person's belonging, although its meaning has changed throughout history. Cooper describes citizenship as the intersection of equality and rights, which relates to integration ethics, differentiation ethics, and fragmentation ethics. Citizenship evolved as people fled war zones or sought better living conditions. This derives from ancient Greek and Roman civilizations, which developed within empires such as Rome rather than modern nation-states. Governments offered citizenship to provide rights and identification, allowing people to live freely and without fear of discrimination. 
Despite its promise, citizenship faces challenges in many countries. For example, dual citizenship raises questions of loyalty as citizens are expected to align with one nation’s cultural and political goals. The understanding of citizenship has varied among cultures as well. In medieval Europe, citizenship was linked to cities rather than nations, but monarchies and empires gradually redefined it. The British Empire, for instance, treated immigrants as part of the kingdom; however, rights were unequally distributed, and colonized people faced exclusion despite fighting for equality using the empire's ideals. The rise of capitalism further complicated citizenship by linking it to economic roles like property ownership or labor. Although poor workers were nominally considered equal citizens, systemic inequities persisted, sparking debates about fairness. Furthermore, movies show the challenges immigrants face due to citizenship, including deportation, job insecurity, language barriers, and uncertain futures. Religion has also influenced citizenship, often leading to violence and separating populations like the massacres aimed to remove entire Jewish communities. The Rwandan genocide, in the movie, shows how even diplomats, despite being from the same country, were stripped of their citizenship and targeted for extermination due to their perceived disloyalty and the actions of their fellow countrymen.
In conclusion, citizenship is a dynamic and diverse concept that has evolved in response to historical, cultural, and political circumstances.
    '''
label, conf = classify_essay(sample)
print(f"{label}   ({conf:.2%} confidence)")


Human   (66.81% confidence)


In [ ]:


from tensorflow.keras.preprocessing.sequence import pad_sequences

def classify_essay(text):
    stat_p_human = stat_model.predict_proba(tfidf.transform([text]))[0,1]
    seq         = tokenizer.texts_to_sequences([text])
    padded      = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    dl_p_human  = dl_model.predict(padded, verbose=0)[0,0]
    p_human     = (stat_p_human + dl_p_human) / 2

    if p_human > 0.5:
        return "Human", p_human
    else:
        return "AI", 1 - p_human

for sample in [
    '''
    9th January 2025
The Evolution of Citizenship
Citizenship represents a person's belonging, although its meaning has changed throughout history. Cooper describes citizenship as the intersection of equality and rights, which relates to integration ethics, differentiation ethics, and fragmentation ethics. Citizenship evolved as people fled war zones or sought better living conditions. This derives from ancient Greek and Roman civilizations, which developed within empires such as Rome rather than modern nation-states. Governments offered citizenship to provide rights and identification, allowing people to live freely and without fear of discrimination. 
Despite its promise, citizenship faces challenges in many countries. For example, dual citizenship raises questions of loyalty as citizens are expected to align with one nation’s cultural and political goals. The understanding of citizenship has varied among cultures as well. In medieval Europe, citizenship was linked to cities rather than nations, but monarchies and empires gradually redefined it. The British Empire, for instance, treated immigrants as part of the kingdom; however, rights were unequally distributed, and colonized people faced exclusion despite fighting for equality using the empire's ideals. The rise of capitalism further complicated citizenship by linking it to economic roles like property ownership or labor. Although poor workers were nominally considered equal citizens, systemic inequities persisted, sparking debates about fairness. Furthermore, movies show the challenges immigrants face due to citizenship, including deportation, job insecurity, language barriers, and uncertain futures. Religion has also influenced citizenship, often leading to violence and separating populations like the massacres aimed to remove entire Jewish communities. The Rwandan genocide, in the movie, shows how even diplomats, despite being from the same country, were stripped of their citizenship and targeted for extermination due to their perceived disloyalty and the actions of their fellow countrymen.
In conclusion, citizenship is a dynamic and diverse concept that has evolved in response to historical, cultural, and political circumstances.
    '''
]:
    label, conf = classify_essay(sample)
    print(len(sample))
    print(f"{label:>6}   ({conf:.2%} confidence)")


2233
 Human   (66.81% confidence)
